In [1]:
import os

In [2]:
%pwd

'd:\\IT-service_delivery_risk_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\IT-service_delivery_risk_predictor'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Aryasree20/IT-service_delivery_risk_predictor.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Aryasree20"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0675968246bbebab8413556447840afd8aaa8e48"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    plots_dir: Path
    metric_file_name: Path
    model_path: Path
    X_test_path: Path
    y_test_path: Path
    mlflow_uri: str
    all_params: dict

In [7]:
from risk_predictor.constants import *
from risk_predictor.utils.common import read_yaml,create_directories


In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir, config.plots_dir])
        
        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            plots_dir=config.plots_dir,
            metric_file_name=config.metric_file_name,
            model_path=config.model_path,
            X_test_path=config.X_test_path,
            y_test_path=config.y_test_path,
            mlflow_uri="https://dagshub.com/Aryasree20/IT-service_delivery_risk_predictor.mlflow",
            all_params=self.params.KnnClassifier
        )    

In [8]:
import os
import joblib
import mlflow
import mlflow.sklearn
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
    auc
)
from sklearn.preprocessing import label_binarize
from risk_predictor import logger
